In [22]:
import os
import pandas as pd

current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory) + "/"
solList = ["fcfs_solution.csv", "heuristicSol.csv"]

solution = pd.read_csv(parent_directory + solList[0], index_col=0)

def count_using_port(row):
    if (row['charge'] >= 0.41666) | (row['discharge'] >= 0.41666):
        return 2
    elif ((row['charge'] < 0.41666) & (row['charge'] > 0)) | ((row['discharge'] < 0.41666) & (row['discharge'] > 0)):
        return 1
    else:
        return 0
    
def count_using_charger(row):
    if row['port_count'] != 0:
        return 1
    else:
        return 0

def reform_schedule(row):
    if row['consumption'] == 0:
        return False
    else:
        return True
    
solution['port_count'] = solution.apply(count_using_port, axis =1)
solution['charger_count'] = solution.apply(count_using_charger, axis = 1)
solution['operation'] = solution.apply(reform_schedule, axis = 1)
solution['departure'] = solution['operation'].shift(1) != solution['operation']  # True: 값이 바뀜, False: 바뀌지 않음

# 값이 바뀌는 순간을 가지는 행만 선택
departure_point = solution[solution['departure']]
# departure_point = departure_point[::2]
departure_point = departure_point[departure_point['operation'] == True]

In [24]:
### 포트 제약
port_count = solution.groupby('period').sum()[['port_count']]
charger_count = solution.groupby('period').sum()[['charger_count']]
_p =charger_count[charger_count['charger_count'] > 30]
_c =port_count[port_count['port_count'] > 60]

if _p.empty & _c.empty:
    print(f"\nPassed port resource constraint")


Passed port resource constraint


In [30]:
count = 0
problem_point = []
for row in departure_point.itertuples():
    if row.bus[0] != "N":
        required = 250 * 0.7
    else:
        required = 350 * 0.7
    
    if row.SOC < required:
        count += 1
        problem_point.append(row)
    else:
        pass

problem_df = pd.DataFrame(problem_point)

if count != 0:
    print("Failed rorc constraint\n")
    print(problem_df)
else:
    print("Passed rorc constraint")

Failed rorc constraint

    Index    bus  period  charge  discharge  consumption         SOC  \
0   15450  150_1     515     0.0        0.0         0.25  174.416668   
1   24090  150_1     803     0.0        0.0         0.25  174.416670   
2   32730  150_1    1091     0.0        0.0         0.25  174.416672   
3   41370  150_1    1379     0.0        0.0         0.25  174.416674   
4   24721  150_2     824     0.0        0.0         0.25  171.833337   
..    ...    ...     ...     ...        ...          ...         ...   
57  32508  160_6    1083     0.0        0.0         0.25  174.750005   
58  33348  160_6    1111     0.0        0.0         0.25  169.750005   
59  41148  160_6    1371     0.0        0.0         0.25  174.750007   
60  41988  160_6    1399     0.0        0.0         0.25  169.750007   
61  41840  160_8    1394     0.0        0.0         0.25  174.333341   

    port_count  charger_count  operation  departure  
0            0              0       True       True  
1  

In [32]:
# 모든 행을 출력
pd.set_option('display.max_rows', None)
problem_df

,Index,bus,period,charge,discharge,consumption,SOC,port_count,charger_count,operation,departure
0,15450,150_1,515,0.0,0.0,0.25,174.416668,0,0,True,True
1,24090,150_1,803,0.0,0.0,0.25,174.416670,0,0,True,True
2,32730,150_1,1091,0.0,0.0,0.25,174.416672,0,0,True,True
3,41370,150_1,1379,0.0,0.0,0.25,174.416674,0,0,True,True
4,24721,150_2,824,0.0,0.0,0.25,171.833337,0,0,True,True
5,32521,150_2,1084,0.0,0.0,0.25,174.333338,0,0,True,True
6,33361,150_2,1112,0.0,0.0,0.25,169.333338,0,0,True,True
7,41161,150_2,1372,0.0,0.0,0.25,172.666674,0,0,True,True
8,42001,150_2,1400,0.0,0.0,0.25,167.666674,0,0,True,True
9,6932,150_3,231,0.0,0.0,0.25,174.749999,0,0,True,True
